# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv", encoding = "utf-8")
weather_data.head()

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
0,Clyde River,70.4692,-68.5914,-20.20,76,75,13.80,CA,1612480227
1,Mataura,-46.1927,168.8643,65.01,51,42,3.80,NZ,1612480260
2,Saa,4.3667,11.4500,73.26,87,11,4.76,CM,1612480260
3,St Albans,51.7500,-0.3333,46.00,93,75,8.05,GB,1612479968
4,Rikitea,-23.1203,-134.9692,78.85,68,97,10.94,PF,1612480261


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
city_locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

In [5]:
fig = gmaps.figure(zoom_level=1.9, center=(25,0))

heat_layer = gmaps.heatmap_layer(city_locations,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# reduce dataframe to include only cities matching a certain criteria
# max temp>=75, <85
# wind speed<5mph
# cloudiness<= 5

reduced_wd = weather_data.loc[(weather_data["Max Temp. (F)"]>=75) &
                             (weather_data["Max Temp. (F)"]<85) &
                             (weather_data["Wind Speed (mph)"]<5) &
                              (weather_data["Cloudiness (%)"]<=15)]
reduced_wd

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC)
280,Banfora,10.6333,-4.7667,78.24,18,3,2.93,BF,1612480365
374,Port Macquarie,-31.4333,152.9167,82.00,71,15,1.01,AU,1612480346
383,Santa Fe,-31.6333,-60.7000,79.00,53,1,1.99,AR,1612480363
465,Rafaela,-31.2503,-61.4867,78.01,44,0,1.99,AR,1612480429
471,Adjumani,3.3779,31.7909,75.61,29,5,3.22,UG,1612480151
529,Maxixe,-23.8597,35.3472,79.92,85,4,1.72,MZ,1612480454
567,Coyhaique,-45.5752,-72.0662,82.40,30,0,4.61,CL,1612480213
577,Inhambane,-23.8650,35.3833,80.76,83,3,1.99,MZ,1612480392


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = reduced_wd.copy()
hotel_df["Hotel Name"] = ""
hotel_df.dropna()
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
280,Banfora,10.6333,-4.7667,78.24,18,3,2.93,BF,1612480365,
374,Port Macquarie,-31.4333,152.9167,82.00,71,15,1.01,AU,1612480346,
383,Santa Fe,-31.6333,-60.7000,79.00,53,1,1.99,AR,1612480363,
465,Rafaela,-31.2503,-61.4867,78.01,44,0,1.99,AR,1612480429,
471,Adjumani,3.3779,31.7909,75.61,29,5,3.22,UG,1612480151,
529,Maxixe,-23.8597,35.3472,79.92,85,4,1.72,MZ,1612480454,
567,Coyhaique,-45.5752,-72.0662,82.40,30,0,4.61,CL,1612480213,
577,Inhambane,-23.8650,35.3833,80.76,83,3,1.99,MZ,1612480392,


In [8]:
# https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters
params = {"radius": 5000,
         "types": "lodging",
          "key":g_key}
          
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
#     print(hotel_name.url)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
hotel_df

,City Name,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date (UTC),Hotel Name
280,Banfora,10.6333,-4.7667,78.24,18,3,2.93,BF,1612480365,Residences David Hebié
374,Port Macquarie,-31.4333,152.9167,82.00,71,15,1.01,AU,1612480346,Rydges Port Macquarie
383,Santa Fe,-31.6333,-60.7000,79.00,53,1,1.99,AR,1612480363,Conquistador Hotel
465,Rafaela,-31.2503,-61.4867,78.01,44,0,1.99,AR,1612480429,Parra Hotel & Suites
471,Adjumani,3.3779,31.7909,75.61,29,5,3.22,UG,1612480151,Hotel Zawadi
529,Maxixe,-23.8597,35.3472,79.92,85,4,1.72,MZ,1612480454,Oceano
567,Coyhaique,-45.5752,-72.0662,82.40,30,0,4.61,CL,1612480213,Diego De Almagro Coyhaique
577,Inhambane,-23.8650,35.3833,80.76,83,3,1.99,MZ,1612480392,Oceano


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))